# Generator 란?
Generator 란 컴퓨터 공학적 의미로써 다음 의미를 가지고 있다.
> In computer science, a **generator** is a routine that can be used to control the iteration behaviour of a loop. - [wikipedia](https://en.wikipedia.org/wiki/Generator_(computer_programming))
>

파이썬에선 Generator 란 **"iterator 를 생성해주는 개념 또는 함수"** 이다. 
- 파이썬에서 generator는 지속적인 상태를 유지하는 기능에 용이하다. <br>
  => 이는 증가하는 계산량과 반복에 유용하다.
- Generator 는 메모리를 효율적으로 사용하기 위해 배열 대신해 사용한다.<br>
  => 제너레이터는 Iterable Object 를 **lazily** 생성한다.

#### Iterable Object 를 **lazily** 생성?
생성한 Iterable Object 에서 i번째 값이 필요할 때까지 생성되지 않는다. => 가상메모리에 먼저 할당되지 않는다.<br>
> Generators do not store the values, but rather the computation logic with the state of the function, similar to an unevaluated function instance ready to be fired. - [Anuradha Wickramarachchi](https://medium.com/towards-data-science/python-generators-393455aa48a3)
>

Generator를 만드는 방법은 대표적으로 2 가지이다.
## 1. Generator Comprehension
이전에 [파이썬 List Comprehension](https://github.com/JihoJu/Python/blob/main/Comprehension/List_Comprehension.ipynb)에 대해 정리를 했다. Comprehension은 List 뿐만 아니라 set, tuple, dict으로 확장이 가능했다. <br>
Tuple Comprehension 식에서 tuple()을 사용한 방법이 아니라 (n for n in range(10)) 방법으로 comprehension 식을 쓰면 다음과 같은 결과가 나온다.

In [121]:
print((n for n in range(10)))

<generator object <genexpr> at 0x2dd348040>


**Generator Comprehension**은 리스트 생성 방식 **[ ]** -> **( )** 로 대신해서 사용한다.<br>
배열과 달리, generator는 runtime 시 값을 생성한다.

In [7]:
import sys
some_list = [i for i in range(10000)]
some_generator = (i for i in range(10000))

print(sys.getsizeof(some_list)) # some_list: [0, 1, 2, ..., 9999] 출력
print(sys.getsizeof(some_generator), some_generator)

85176
112 <generator object <genexpr> at 0x106469cf0>


위의 코드 결과를 보면, Generator가 List에 비해 메모리를 매우! 덜 차지하는 것을 확인할 수 있다.

## 2. yield 문 사용
**return 문 대신 yield 문을 사용하여 generator를 표현한 방법이다.** <br>
다음 예제는 1부터 무한 루프를 돌면서 해당 값이 소수라면 generate 하는 코드이다.

In [100]:
def is_prime(n):
    if n < 2 or n % 1 > 0:
        return False
    elif n == 2 or n == 3:
        return True
    for x in range(2, int(n**0.5) + 1):
        if n % x == 0:
            return False
    return True

def get_prime():
    value = 0
    while True:
        if is_prime(value):
            yield value
        value += 1

In [20]:
primes = get_prime()
print(primes)
print(next(primes))
print(next(primes))
print(next(primes))
print(next(primes))
print(len(primes))

<generator object get_prime at 0x1091179e0>
2
3
5
7


TypeError: object of type 'generator' has no len()

get_prime() 함수를 호출하면 generator 객체가 생성 후 primes 변수에 할당된다.<br>
primes 변수는 generator 이기에 다음 특징을 갖는다.
- **generator는 iterable object를 lazy하게 생성해서 미리 length를 알 수 없다.** <br>
=> 위의 코드에서 len(primes) 시 Error가 발생한다.

### 왜 yield를 사용해야할까??
다음 상황을 가정해보자.
한 log file의 크기가 1TB이고 말뭉치가 500000 이다.<br>
이 file 의 log data를 모두 읽은 후 parsing 후의 결과를 또다른 file에 즉시 저장한다고 가정해보자<br>

1. File 의 모든 데이터를 읽은 후 list data type에 데이터를 할당한 후 parsing을 진행한다.
2. 한번에 file을 open할 필요없이 1줄씩 읽은 후 parsing 후 yield로 결과를 반환 받는다.

In [79]:
# 1번째 방법
def read_data_1(path):
    with open(path) as f:
        data = [d for d in f.readlines()]
    return data

def parse(data):
    line = data.strip().lower()
    return line, len(data)

def function_1():
    for data in read_data_1('./hadoop.log'):
        d, s = parse(data)
        # print(parse(d))

In [80]:
# 2번째 방법 - Generator(yield) 사용
def read_data_2(path):
    with open(path) as f:
        for data in f:
            yield data.strip().lower(), len(data.strip().lower())
            
def function_2():
    data2 = read_data_2("./hadoop.log")
    for l, s in data2:
        continue
        # print(l, s)

In [81]:
import time

def benchmark(function, function_name):
    start = time.time()
    function()
    end = time.time()
    print("{0} seconds for {1}".format(round(end-start, 2), function_name))

benchmark(function_1, "function_1")
benchmark(function_2, "function_2")

0.01 seconds for function_1
0.01 seconds for function_2


1번 방법의 문제점은 한번에 file을 읽어 1TB만큼의 메모리를 계속 할당해야만 한다는 점이다.<br>
2번 방법은 **한번에 file을 읽지 않고 각 iter마다 1줄씩 읽는다.**
=> yield를 사용한 2번 방법이 메모리 측면에서 매우 효율적이다.

또한, 2번 방법은 **clean code 문법을 제공**한다.

## 그럼 Generator 2가지 방법 중 어떤 방법을 사용해야할까?
앞에서 Generator를 만드는 방법으로 2가지 방법을 정리했다. 그럼 이 중 어떤 방법을 사용하는게 좋을까? <br>
=> 답은 **'상황에 따라 다르다'** 이다. <br><br>

List Comprehension으로 무조건 한 줄로 표현하는 것이 가독성, 확장성, 유지보수성 등이 비효율적인 상황이 있었다. <br>
Generator Comprehension에서도 마찬가지이다. 어떤 기능이든 **'trade-off'** 가 존재하기 마련이다. <br>
Generator Comprehension 사용하기 더 쉬워지고 간략해지는 대신 복잡한 요구상항을 매우 비효율적으로 표현해야만 한다.(기능 한계) <br>
### 결론
**복잡한 프로그램일수록 Iterable, Iterator를 만들어 쓰는 yield 방식이 좋을 선택일수도 있고, 프로그램이 매우 단순하다면 Generator Comprehension을 사용하는 방법이 좋은 선택이 될 수 있을 것이다.**

### Generator Comprehension가 효율적인 예제
다음 상황을 가정해보자.<br>
1부터 10까지의 값의 제곱을 모두 더한 값을 구한다고 가정해보자. 이를 코드로 나타내면 다음과 같다.

In [94]:
# list comprehension
def list_func():
    print(sum([n ** 2 for n in range(1, 11)]))

# generator comprehension
def generator_func():
    print(sum((n ** 2 for n in range(1, 11))))
    

def benchmark(function, function_name):    
    start = time.time()
    function()
    end = time.time()
    print("{0} seconds for {1}".format(round(end - start, 10), function_name))
    
benchmark(list_func, "list_func")
benchmark(generator_func, "generator_func")

385
0.0001547337 seconds for list_func
385
1.50204e-05 seconds for generator_func


위의 경우 generator comprehension이 메모리, 속도 측면에서 더욱 효율적인 것을 확인할 수 있다.

## send, throw, and close

### send
위의 prime number 예제를 생각해보자. 만약 10 이상의 소수, 100 이상의 소수를 바로 구하고 싶으면 어떻게 하면 될까?<br>
send() 함수를 사용하면 된다!!

In [117]:
# send 를 위한 get_prime 함수
def get_prime_2():
    value = 0
    while True:
        if is_prime(value):
            i = yield value # next(primes) 시 여기서 일단 멈추고 send 로 10을 받아오면 i에 할당
            if i is not None:
                value = i
        value += 1

primes = get_prime_2()
print(next(primes))
print(primes.send(10))
print(primes.send(100))

primes = get_prime_2()
print(primes.send(10))
print(primes.send(100))

2
11
101


TypeError: can't send non-None value to a just-started generator

**NOTE**: send() 함수를 호출하기 전에 적어도 한번은 next() 함수를 호출해야만 한다. 그렇지 않으면 위와 같은 에러가 발생!! 

### throw function
만약 get_prime 함수에서 10 이상의 값이 나오면 overflow or timeout이 된다고 가정해보자.<br>
**throw() 함수**는 generator 가 예외를 raising 하여 멈추게 해준다.

In [118]:
primes = get_prime_2()
for x in primes:
    if x > 10:
        primes.throw(ValueError, "Too large")
    print(x)

2
3
5
7


ValueError: Too large

### close function
**close() 함수**는 throw() function과 달리 예외 처리 없이 generator 를 종료시킬 수 있다.

In [119]:
primes = get_prime_2()
for x in primes:
    if x > 10:
        primes.close()
    print(x)

2
3
5
7
11


=> 11 값까지 나오는 이유는 이는 c/c++에서 do while 과 같이 작동하기 때문!!!

## List vs Generator

결과만 따져본다면 List Comprehension를 쓰거나, 함수를 만들어 list를 반환하는 등과 같은 방법을 써도 결과는 같다. 굳이 왜 generator를 사용해야할까??<br>
다음 2개의 코드를 비교해보자.

In [99]:
# 1. list를 사용해 출력
for elem in [n for n in range(1, 11)]:
    print(elem)
    
# 2. Generator를 사용해 출력
for elem in (n for n in range(1, 11)):
    print(elem)

두 개의 결과는 똑같으면서 표현식도 비슷하다. 하지만, 이 두 방법의 가장 큰 차이점은 **메모리**이다.<br>
#### 1. list comprehension
리스트를 선언하면 메모리에 해당 리스트 객체에 대한 메모리가 할당되고 리스트의 주소값을 반환한다. 즉, 1\~10의 원소값을 저장해두기 위한 메모리 공간이 할당된다.<br> 
=> **eager loading pattern**
#### 2. generator comprehension
generator는 지정한 규칙대로 값을 반환할 규칙과 현재 어디까지 반환했는지 등을 관리하는 여러 상태값을 담고 있지만 generator 생성 당시  1\~10까지의 모든 값을 메모리에 할당해두지 않는다.<br>
=> **lazy loading** pattern <br><br>
여기에 결정적인 차이가 있다!!!

### 그런데도 왜 이 두 개를 분리해서 사용할까?
라고 생각할 수 있다. 다음 코드를 비교해보자.

In [97]:
big_list = [i for i in range(1, 100000000000000000000+1)]

# 생성이 되지 않는다.

In [98]:
big_generator = (i for i in range(1, 100000000000000000000+1))

# 문제없이 바로 생성됨.

list comprehension으로 매우 큰 list를 생성하게 되면 아마 멈추거나 인터프리터가 종료될 것이다.<br>
**그 이유는 list 자료구조는 eager loading 으로 원소의 크기에 따라 총 크기가 결정되므로 매우 큰 크기의 list를 선언하면, 일반 컴퓨터의 한정된 용량의 메모리로는 감당할 수 없기 때문이다.**
반면에, generator comprehension으로 생성한 매우 큰 generator는 문제없이 실행된다.<br>

# 결론
앞으로 크기가 매우 큰 자료구조를 다룰 일이 많을 것이다. 예를 들어, 매운 큰 파일을 읽는다거나, 네트워크를 통해 대용량의 데이터를 다운로드 등. 이러한 대용량의 데이터를 한 줄 등의 한 단위를 필요로 할 때 한 줄씩 평가하는(evaluate) lazy loading이 한 번에 모든 데이터를 loading 하는 eager loading 보다 안전하고 효율적이다. <br><br>

**Generator의 장점은 일련의 값을 사용할 때 모든 값을 메모리에 모두 로딩하는 대신 한 값씩 필요할 때마다 로딩, 평가함으로써 메모리를 절약하고, 메모리 부족으로 프로그램이 실패하는 것을 방지할 수 있는 안전성을 제공한다.**

# References
- https://medium.com/towards-data-science/pythons-list-generators-what-when-how-and-why-2a560abd3879
- https://shoark7.github.io/programming/python/iterable-iterator-generator-in-python#4a
- https://medium.com/towards-data-science/python-generators-393455aa48a3